In [38]:
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full,y_train_full),(X_test,y_test)=fashion_mnist.load_data()
print(X_train_full.shape)
X_train_full,X,y_train_full,Y=X_train_full[5000:],X_train_full[:5000],y_train_full[5000:],y_train_full[:5000]
X_test = X_test / 255.0
x_valid,x_train=X_train_full[:5000]/255.0,X_train_full[5000:]/255.0
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]

class_names=["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
 "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

(60000, 28, 28)


In [40]:
X_test.shape

(10000, 28, 28)

In [41]:
X_test,XT,y_test,YT=X_test[2000:],X_test[:2000],y_test[2000:],y_test[:2000]

In [37]:
X_test.shape,y_test.shape,X_train_full.shape,y_train_full.shape

((8000, 28, 28), (8000,), (55000, 28, 28), (55000,))

In [67]:
def generate_siamese_batch(X, y, batch_size=64):
    X1, X2, y_batch = [], [], []
    num_classes = np.unique(y).shape[0]
    class_indices = {label: np.where(y == label)[0] for label in range(num_classes)}
    for _ in range(batch_size // 2):
        # Same class pair
        label = np.random.randint(0, num_classes)
        idx1, idx2 = np.random.choice(class_indices[label], 2, replace=False)
        X1.append(X[idx1])
        X2.append(X[idx2])
        y_batch.append(0)

        # Different class pair
        label1, label2 = np.random.choice(range(num_classes), 2, replace=False)
        idx1 = np.random.choice(class_indices[label1])
        idx2 = np.random.choice(class_indices[label2])
        X1.append(X[idx1])
        X2.append(X[idx2])
        y_batch.append(1)

    return np.array(X1), np.array(X2), np.array(y_batch)


In [145]:
X1_batch, X2_batch, y_batch = generate_siamese_batch(x_train, y_train, batch_size=30000)
X1_validbatch, X2_validbatch, y_validbatch = generate_siamese_batch(x_valid, y_valid, batch_size=800)
X1_testbatch, X2_testbatch, y_testbatch = generate_siamese_batch(X_test, y_test, batch_size=300)

In [146]:
X1_batch.shape,X2_batch.shape, y_batch.shape

((30000, 28, 28), (30000, 28, 28), (30000,))

In [147]:
inputA=keras.layers.Input(shape=[28,28])
flatA = keras.layers.Flatten()(inputA)
hiddenA1=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(flatA)
hiddenA2=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenA1)
hiddenA3=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenA2)
hiddenA4=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenA3)
hiddenA5=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenA4)
out_a=keras.layers.Dense(10,activation='elu')(hiddenA5)

In [148]:
inputB=keras.layers.Input(shape=[28,28])
flatB = keras.layers.Flatten()(inputB)
hiddenB1=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(flatB)
hiddenB2=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenB1)
hiddenB3=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenB2)
hiddenB4=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenB3)
hiddenB5=keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hiddenB4)
out_b=keras.layers.Dense(10,activation='elu')(hiddenB5)

In [149]:
merged = keras.layers.Concatenate()([out_a, out_b])
hidden = keras.layers.Dense(10, activation='elu',kernel_initializer="he_normal")(merged)
output = keras.layers.Dense(1, activation='sigmoid')(hidden)

model = keras.Model(inputs=[inputA, inputB], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [150]:
model.fit((X1_batch, X2_batch),y_batch,validation_data=((X1_validbatch, X2_validbatch),y_validbatch),epochs=30)

Epoch 1/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7353 - loss: 0.5079 - val_accuracy: 0.8288 - val_loss: 0.3903
Epoch 2/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8434 - loss: 0.3624 - val_accuracy: 0.8487 - val_loss: 0.3437
Epoch 3/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8664 - loss: 0.3218 - val_accuracy: 0.8637 - val_loss: 0.3270
Epoch 4/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8731 - loss: 0.3060 - val_accuracy: 0.8600 - val_loss: 0.3170
Epoch 5/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8801 - loss: 0.2972 - val_accuracy: 0.8675 - val_loss: 0.3036
Epoch 6/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8807 - loss: 0.2935 - val_accuracy: 0.8775 - val_loss: 0.3039
Epoch 7/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8813 - loss: 0.2917 - val_accuracy: 0.8725 - val_loss: 0.3122
Epoch 8/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8867 - loss: 0.2807 - val_accuracy: 0

In [151]:
model.evaluate((X1_testbatch, X2_testbatch), y_testbatch)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8754 - loss: 0.3738  


[0.3457978069782257, 0.8833333253860474]

Using same weights shared networks a and b 

In [152]:
input_shape = (28, 28)
base_input = keras.layers.Input(shape=input_shape)
flatten = keras.layers.Flatten()(base_input)
hidden1 = keras.layers.Dense(100, kernel_initializer="he_normal", activation='elu')(flatten)
hidden2 = keras.layers.Dense(100, kernel_initializer="he_normal", activation='elu')(hidden1)
hidden3 = keras.layers.Dense(100, kernel_initializer="he_normal", activation='elu')(hidden2)
hidden4 = keras.layers.Dense(100, kernel_initializer="he_normal", activation='elu')(hidden3)
hidden5 = keras.layers.Dense(100,kernel_initializer="he_normal",activation='elu')(hidden3)
output_embedding =keras.layers.Dense(10,activation='elu')(hidden5)
base_network = keras.models.Model(inputs=base_input, outputs=output_embedding)

In [153]:
inputa = keras.layers.Input(shape=input_shape)
inputb = keras.layers.Input(shape=input_shape)

processed_a = base_network(inputa) 
processed_b = base_network(inputb) 

In [154]:
concat=keras.layers.Concatenate()([processed_a,processed_b])
hiddenN = keras.layers.Dense(10, activation='elu',kernel_initializer="he_normal")(concat)
outputN = keras.layers.Dense(1, activation='sigmoid')(hiddenN)

modelN = keras.Model(inputs=[inputa,inputb], outputs=outputN)
modelN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [155]:
modelN.fit((X1_batch, X2_batch),y_batch,validation_data=((X1_validbatch, X2_validbatch),y_validbatch),epochs=30)

Epoch 1/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7432 - loss: 0.4856 - val_accuracy: 0.8675 - val_loss: 0.3263
Epoch 2/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8651 - loss: 0.3162 - val_accuracy: 0.8625 - val_loss: 0.3120
Epoch 3/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8764 - loss: 0.2872 - val_accuracy: 0.8800 - val_loss: 0.2936
Epoch 4/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8944 - loss: 0.2561 - val_accuracy: 0.8963 - val_loss: 0.2762
Epoch 5/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9028 - loss: 0.2370 - val_accuracy: 0.8900 - val_loss: 0.2788
Epoch 6/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9031 - loss: 0.2271 - val_accuracy: 0.9075 - val_loss: 0.2389
Epoch 7/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9137 - loss: 0.2101 - val_accuracy: 0.9013 - val_loss: 0.2443
Epoch 8/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9169 - loss: 0.2046 - val_accuracy: 0.

In [156]:
modelN.evaluate((X1_testbatch, X2_testbatch), y_testbatch)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9001 - loss: 0.4570  


[0.37647515535354614, 0.8999999761581421]

In [157]:
tshirt_idx = np.where(y_test == 0)[0][0] # T-shirt
trouser_idx = np.where(y_test == 1)[0][0] #Trouser
# Add a batch dimension to each image
image_A = np.expand_dims(X_test[tshirt_idx], axis=0)  # Shape becomes (1, 28, 28)
image_B = np.expand_dims(X_test[trouser_idx], axis=0) 
prediction=model.predict([image_A,image_B])
print(prediction[0][0]*100)
if prediction[0][0] < 0.5:
    print("The two images are predicted to be SIMILAR.")
else:
    print("The two images are predicted to be DISSIMILAR.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
99.99963045120239
The two images are predicted to be DISSIMILAR.


In [158]:

tshirt_idx2 = np.where(y_test == 0)[0][1] # T-shirt

# Add a batch dimension to each image
image_A = np.expand_dims(X_test[tshirt_idx], axis=0)  # Shape becomes (1, 28, 28)
image_B = np.expand_dims(X_test[tshirt_idx2], axis=0) 
prediction=model.predict([image_A,image_B])
print(prediction[0][0]*100)
if prediction[0][0] < 0.5:
    print("The two images are predicted to be SIMILAR.")
else:
    print("The two images are predicted to be DISSIMILAR.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1.7256850376725197
The two images are predicted to be SIMILAR.


In [159]:
model.save('fashion_minist_image_comparision_seperateA&Bweights.h5')
modelN.save('fashion_minist_image_comparision_sameA&weights.h5')